In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/code-custome-phogpt-4b/BTEC_gemini_rag_1000.json
/kaggle/input/code-custome-phogpt-4b/Cusome_PhoGPT_4B/hf_prefixlm_converter.py
/kaggle/input/code-custome-phogpt-4b/Cusome_PhoGPT_4B/norm.py
/kaggle/input/code-custome-phogpt-4b/Cusome_PhoGPT_4B/flash_attn_triton.py
/kaggle/input/code-custome-phogpt-4b/Cusome_PhoGPT_4B/attention.py
/kaggle/input/code-custome-phogpt-4b/Cusome_PhoGPT_4B/meta_init_context.py
/kaggle/input/code-custome-phogpt-4b/Cusome_PhoGPT_4B/fc.py
/kaggle/input/code-custome-phogpt-4b/Cusome_PhoGPT_4B/custom_embedding.py
/kaggle/input/code-custome-phogpt-4b/Cusome_PhoGPT_4B/param_init_fns.py
/kaggle/input/code-custome-phogpt-4b/Cusome_PhoGPT_4B/configuration_mpt.py
/kaggle/input/code-custome-phogpt-4b/Cusome_PhoGPT_4B/adapt_tokenizer.py
/kaggle/input/code-custome-phogpt-4b/Cusome_PhoGPT_4B/blocks.py
/kaggle/input/code-custome-phogpt-4b/Cusome_PhoGPT_4B/warnings.py
/kaggle/input/code-custome-phogpt-4b/Cusome_PhoGPT_4B/__init__.py
/kaggle/input/code-custome-ph

In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U datasets
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U loralib
!pip install -q -U einops
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 4.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 r

In [3]:
import json
import os
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
import shutil
from pathlib import Path

from pprint import pprint
from tqdm import tqdm
from datasets import load_dataset
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer

import warnings


2025-05-29 10:35:16.443740: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748514916.650107      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748514916.714144      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
REVISION = "e5dd0ad1a5eed58e077c1b799f1d54790b40a507"
cache_dir = f"/root/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B/{REVISION}"

# Tạo thư mục cache
os.makedirs(cache_dir, exist_ok=True)
print(f"📁 Created cache directory: {cache_dir}")

# OPTION 1: Copy từ Kaggle input (nếu bạn upload dataset)
input_dir = "/kaggle/input/code-custome-phogpt-4b/Cusome_PhoGPT_4B"  # Thay tên dataset của bạn

print("📥 Copying files from Kaggle input...")

# List files trong input
input_files = os.listdir(input_dir)
print(f"Available files: {input_files}")

# Copy tất cả files
for file in input_files:
    src = os.path.join(input_dir, file)
    dst = os.path.join(cache_dir, file)

    if os.path.isfile(src):
        shutil.copy2(src, dst)
        print(f"✅ Copied {file}")

📁 Created cache directory: /root/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B/e5dd0ad1a5eed58e077c1b799f1d54790b40a507
📥 Copying files from Kaggle input...
Available files: ['hf_prefixlm_converter.py', 'norm.py', 'flash_attn_triton.py', 'attention.py', 'meta_init_context.py', 'fc.py', 'custom_embedding.py', 'param_init_fns.py', 'configuration_mpt.py', 'adapt_tokenizer.py', 'blocks.py', 'warnings.py', '__init__.py', 'modeling_mpt.py', 'ffn.py']
✅ Copied hf_prefixlm_converter.py
✅ Copied norm.py
✅ Copied flash_attn_triton.py
✅ Copied attention.py
✅ Copied meta_init_context.py
✅ Copied fc.py
✅ Copied custom_embedding.py
✅ Copied param_init_fns.py
✅ Copied configuration_mpt.py
✅ Copied adapt_tokenizer.py
✅ Copied blocks.py
✅ Copied warnings.py
✅ Copied __init__.py
✅ Copied modeling_mpt.py
✅ Copied ffn.py


In [5]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# Tắt wandb để tránh lỗi API key
os.environ["WANDB_DISABLED"] = "true"
warnings.filterwarnings('ignore')

# Định nghĩa model name và revision
MODEL_NAME = "vinai/PhoGPT-4B"
REVISION = "e5dd0ad1a5eed58e077c1b799f1d54790b40a507"

# Cấu hình quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    revision=REVISION,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

configuration_mpt.py:   0%|          | 0.00/16.4k [00:00<?, ?B/s]

modeling_mpt.py:   0%|          | 0.00/32.4k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/7.38G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/91.0 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/844k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

In [6]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0

    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
    )

In [7]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj",
        "up_proj",
        "o_proj",
        "k_proj",
        "down_proj",
        "gate_proj",
        "v_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 15728640 || all params: 1891866624 || trainables%: 0.8313820752725537


In [8]:
dataset = load_dataset("json", data_files="/kaggle/input/dataset-tuyen-sinh/data_sinh.json")["train"]


Generating train split: 0 examples [00:00, ? examples/s]

In [9]:
def preprocess(example):
    # Định dạng prompt: Giống kiểu Alpaca / ChatGPT
    prompt = f"### Hỏi:\n{example['instruction']}\n\n### Trả lời:\n{example['response']}"

    tokenized = tokenizer(
        prompt,
        padding=True,
        truncation=True
    )
    # labels = input_ids để mô hình học dự đoán các token tiếp theo
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(preprocess)


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [10]:

# Thiết lập tham số huấn luyện
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=2,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="/kaggle/working/experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)


# Huấn luyện bằng SFTTrainer
trainer = SFTTrainer(
    model=model,
    # tokenizer=tokenizer,
    train_dataset=tokenized_dataset,
    args=training_args,
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Truncating train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
1,4.574200
2,4.591300
3,4.664400
4,4.702600
5,6.260200
6,5.623900
7,3.736300
8,4.029400
9,4.760000
10,4.706900


TrainOutput(global_step=1000, training_loss=0.5264212908148765, metrics={'train_runtime': 1797.3309, 'train_samples_per_second': 2.226, 'train_steps_per_second': 0.556, 'total_flos': 5302259261300736.0, 'train_loss': 0.5264212908148765})